In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!pip install datasets
!pip install transformers
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 48.5 MB/s eta 0:00

### Imports

In [3]:
import requests
import json
import torch
import os
from tqdm import tqdm
import sys
import random
from copy import deepcopy
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

### Download Dataset


In [4]:
from datasets import load_dataset

squad = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

### Read Data, Adding Noise and Preprocessing

In [5]:
def read_data(squad, split):

  contexts = []
  questions = []
  answers = []

  for data in squad[split]:
    contexts.append(data['context'])
    questions.append(data['question'])
    answers.append(data['answers'])

  return contexts, questions, answers

In [6]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

# Noise funcs 

# FOR QA ONLY
char_action = ['insert',
        'substitute',
        'delete',
        'swap',
]

def get_action(type):
    return random.choice(char_action)

def add_noise(question, p=0.7):
    """
    Augment a tweet with character-level and word-level noise.

    Args:
        tweet (str): The original tweet.
        p (float): The probability of applying the char level augmentation.

    Returns:
        str: The augmented question.
    """
    # Define a list of character-level augmentation techniques
    char_augmenters = [
        nac.KeyboardAug(aug_char_p=0.2, aug_word_p=0.2, include_special_char=False, include_numeric=False),
        nac.RandomCharAug(action=get_action("char"), aug_char_p=0.1, aug_word_p=0.1),
    ]

    # Define a list of word-level augmentation techniques
    word_augmenters = [
        naw.SpellingAug(),
        naw.SynonymAug(),
    ]

    # Randomly apply a character-level or word-level augmentation with probability p
    if random.random() < p:
        aug = random.choice(char_augmenters)
        noisy_text = aug.augment(question)
    else:
        aug = random.choice(word_augmenters)
        noisy_text = aug.augment(question)
      
    return noisy_text[0]


In [7]:
# train validation splits
train_contexts, train_questions, train_answers = read_data(squad, 'train')
valid_contexts, valid_questions, valid_answers = read_data(squad, 'validation')


# Make False if no noise to be added
noisy = True
noise_percent = 0.2
if noisy:

  for i, question in enumerate(train_questions):

    if random.random() < noise_percent:
      noisy_question = add_noise(question)
      train_questions[i] = noisy_question

  for i, question in enumerate(valid_questions):

    if random.random() < noise_percent:
      noisy_question = add_noise(question)
      valid_questions[i] = noisy_question

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
import random
train_size = int(0.2 * len(squad['train']))
train_indices = random.sample(range(len(squad['train'])), train_size)
train_contexts = []
train_questions = []
train_answers = []

for idx in train_indices:
  group = squad['train'][idx]
  train_contexts.append(group['context'])
  train_questions.append(group['question'])
  train_answers.append(group['answers'])

In [9]:
import random
valid_size = int(0.2 * len(squad['validation']))
valid_indices = random.sample(range(len(squad['validation'])), valid_size)
valid_contexts = []
valid_questions = []
valid_answers = []

for idx in valid_indices:
  group = squad['validation'][idx]
  valid_contexts.append(group['context'])
  valid_questions.append(group['question'])
  valid_answers.append(group['answers'])

In [10]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    end_idx = []
    end_idx.append(start_idx + len(gold_text))

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx[0]] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters



In [11]:
add_end_idx(train_answers, train_contexts)
add_end_idx(valid_answers, valid_contexts)

### Tokenization


In [12]:
from transformers import BertTokenizerFast

#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
from transformers import ElectraTokenizerFast, ElectraForQuestionAnswering

tokenizer = ElectraTokenizerFast.from_pretrained('google/electra-small-discriminator')



In [13]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True)

In [14]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [15]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] energy transfer can be considered for the special case of systems which are closed to transfers of matter. the portion of the energy which is transferred by conservative forces over a distance is measured as the work the source system does on the receiving system. the portion of the energy which does not do work during the transfer is called heat. [ note 4 ] energy can be transferred between systems in a variety of ways. examples include the transmission of electromagnetic energy via photons, physical collisions which transfer kinetic energy, [ note 5 ] and the conductive transfer of thermal energy. [SEP] the portion of energy which does not do work during the transfer is called what? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [16]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'][0]))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'][0] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})



In [17]:
add_token_positions(train_encodings, train_answers)
add_token_positions(valid_encodings, valid_answers)

In [ ]:
train_encodings['start_positions'][:10]

[20, 101, 138, 11, 39, 136, 126, 103, 128, 11]

In [18]:
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [19]:
train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)

In [20]:
from torch.utils.data import DataLoader

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

### Model definition

In [21]:
from transformers import BertForQuestionAnswering, AutoModel

model = ElectraForQuestionAnswering.from_pretrained('google/electra-base-discriminator')


Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.we

### Training

In [22]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
from transformers import AdamW
from sklearn.metrics import f1_score

N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=2e-5)
best_val_loss = float("inf")
model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

  # Evaluate the model on the validation set
  model.eval()
  val_loss = 0
  predictions = []
  true_labels = []
  with torch.no_grad():
    for batch in valid_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss = outputs[0]
      val_loss += loss.item()
      start_logits = outputs[1]
      end_logits = outputs[2]
      start_preds = start_logits.argmax(dim=1)
      end_preds = end_logits.argmax(dim=1)
      predictions.extend(list(zip(start_preds.cpu().numpy(), end_preds.cpu().numpy())))
      true_labels.extend(list(zip(start_positions.cpu().numpy(), end_positions.cpu().numpy())))
  val_loss /= len(valid_loader)

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    path = '/content/gdrive/MyDrive/NLP Project/models/ELECTRA_QA_NOISY20'
    torch.save(model.state_dict(), path+'/model_parameters.pth')

  print(val_loss,best_val_loss)

  # Set the model back to training mode
  model.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 1095/1095 [26:12<00:00,  1.44s/it, loss=1.18]


1.028739159940777 1.028739159940777


Epoch 2: 100%|██████████| 1095/1095 [26:09<00:00,  1.43s/it, loss=1.23]


1.0067894362417378 1.0067894362417378


Epoch 3: 100%|██████████| 1095/1095 [26:07<00:00,  1.43s/it, loss=0.392]


1.0996942318471752 1.0067894362417378


Epoch 4: 100%|██████████| 1095/1095 [26:09<00:00,  1.43s/it, loss=0.364]


1.1480106220657664 1.0067894362417378


Epoch 5: 100%|██████████| 1095/1095 [26:09<00:00,  1.43s/it, loss=0.448]


1.3155288668279361 1.0067894362417378


### Evaluation

In [25]:
import random
from datasets import load_dataset
from torch.utils.data import DataLoader
from collections import Counter

squad_test = load_dataset("squad")
valid_contexts, valid_questions, valid_answers = read_data(squad_test, 'validation')

valid_size = int(0.2 * len(squad_test['validation']))
valid_indices = random.sample(range(len(squad['validation'])), valid_size)
valid_contexts = []
valid_questions = []
valid_answers = []

for idx in valid_indices:
  group = squad_test['validation'][idx]
  valid_contexts.append(group['context'])
  valid_questions.append(group['question'])
  valid_answers.append(group['answers'])

add_end_idx(valid_answers, valid_contexts)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True)
add_token_positions(valid_encodings, valid_answers)
valid_dataset = SQuAD_Dataset(valid_encodings)
valid_loader = DataLoader(valid_dataset, batch_size=16)

  0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
path = '/content/gdrive/MyDrive/NLP Project/models/ELECTRA_QA_NOISY20'
model.load_state_dict(torch.load(path+'/model_parameters.pth'))

model = model.to(device)
model.eval()

acc = []
em_score = []
f1_score = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

    # Calculate EM score and F1 score
    for i in range(len(start_true)):
        pred_span = tokenizer.decode(input_ids[i][start_pred[i]:end_pred[i] + 1], skip_special_tokens=True)
        true_span = tokenizer.decode(input_ids[i][start_true[i]:end_true[i] + 1], skip_special_tokens=True)
        em_score.append(int(pred_span == true_span))

        # Compute F1 score
        common = Counter(pred_span.split()) & Counter(true_span.split())
        num_common = sum(common.values())
        precision = num_common / max(len(pred_span.split()), 1)
        recall = num_common / max(len(true_span.split()), 1)
        f1 = (2 * precision * recall) / max((precision + recall), 1e-8)
        f1_score.append(f1)


acc = sum(acc) / len(acc)
em_score = sum(em_score) / len(em_score)
f1_score = sum(f1_score) / len(f1_score)

print("\n\nAccuracy:", acc)
print("EM score:", em_score)
print("F1 score:", f1_score)

print("\n\nT/P\tanswer_start\tanswer_end\n")
for i in range(len(start_true)):
  pred_span = tokenizer.decode(input_ids[i][start_pred[i]:end_pred[i]+1], skip_special_tokens=True)
  true_span = tokenizer.decode(input_ids[i][start_true[i]:end_true[i]+1], skip_special_tokens=True)
  print(f"true\t{start_true[i]}\t{end_true[i]}\t{true_span}\n"
        f"pred\t{start_pred[i]}\t{end_pred[i]}\t{pred_span}\n")

100%|██████████| 133/133 [01:02<00:00,  2.12it/s]



Accuracy: 0.7117011278195489
EM score: 0.587038789025544
F1 score: 0.7777495579865539


T/P	answer_start	answer_end

true	175	175	1985
pred	175	175	1985

true	42	44	francis blackburne
pred	42	44	francis blackburne



### Evaluation on Noisy Data

In [27]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

# Noise funcs 

# FOR QA ONLY
char_action = ['insert',
        'substitute',
        'delete',
        'swap',
]

def get_action(type):
    return random.choice(char_action)

def add_noise(question, p=0.7):
    """
    Augment a tweet with character-level and word-level noise.

    Args:
        tweet (str): The original tweet.
        p (float): The probability of applying the char level augmentation.

    Returns:
        str: The augmented question.
    """
    # Define a list of character-level augmentation techniques
    char_augmenters = [
        nac.KeyboardAug(aug_char_p=0.2, aug_word_p=0.2, include_special_char=False, include_numeric=False),
        nac.RandomCharAug(action=get_action("char"), aug_char_p=0.1, aug_word_p=0.1),
    ]

    # Define a list of word-level augmentation techniques
    word_augmenters = [
        naw.SpellingAug(),
        naw.SynonymAug(),
    ]

    # Randomly apply a character-level or word-level augmentation with probability p
    if random.random() < p:
        aug = random.choice(char_augmenters)
        noisy_text = aug.augment(question)
    else:
        aug = random.choice(word_augmenters)
        noisy_text = aug.augment(question)
      
    return noisy_text[0]


In [28]:
import random
from datasets import load_dataset
from torch.utils.data import DataLoader

squad_test = load_dataset("squad")
valid_contexts, valid_questions, valid_answers = read_data(squad_test, 'validation')

import random
random.seed(42)
random_noise = random.uniform(0.05, 0.15)
print(random_noise)

noisy = True
noise_percent = random_noise
if noisy:

  for i, question in enumerate(valid_questions):

    if random.random() < noise_percent:
      noisy_question = add_noise(question)
      valid_questions[i] = noisy_question
      
  print('added noise')

valid_size = int(0.2 * len(squad_test['validation']))
valid_indices = random.sample(range(len(squad['validation'])), valid_size)
valid_contexts = []
valid_questions = []
valid_answers = []

for idx in valid_indices:
  group = squad_test['validation'][idx]
  valid_contexts.append(group['context'])
  valid_questions.append(group['question'])
  valid_answers.append(group['answers'])

add_end_idx(valid_answers, valid_contexts)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True)
add_token_positions(valid_encodings, valid_answers)
valid_dataset = SQuAD_Dataset(valid_encodings)
valid_loader = DataLoader(valid_dataset, batch_size=16)

  0%|          | 0/2 [00:00<?, ?it/s]

0.11394267984578837
added noise


In [29]:
path = '/content/gdrive/MyDrive/NLP Project/models/ELECTRA_QA_NOISY20'
model.load_state_dict(torch.load(path+'/model_parameters.pth'))

model = model.to(device)
model.eval()

acc = []
em_score = []
f1_score = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

    # Calculate EM score and F1 score
    for i in range(len(start_true)):
        pred_span = tokenizer.decode(input_ids[i][start_pred[i]:end_pred[i] + 1], skip_special_tokens=True)
        true_span = tokenizer.decode(input_ids[i][start_true[i]:end_true[i] + 1], skip_special_tokens=True)
        em_score.append(int(pred_span == true_span))

        # Compute F1 score
        common = Counter(pred_span.split()) & Counter(true_span.split())
        num_common = sum(common.values())
        precision = num_common / max(len(pred_span.split()), 1)
        recall = num_common / max(len(true_span.split()), 1)
        f1 = (2 * precision * recall) / max((precision + recall), 1e-8)
        f1_score.append(f1)


acc = sum(acc) / len(acc)
em_score = sum(em_score) / len(em_score)
f1_score = sum(f1_score) / len(f1_score)

print("\n\nAccuracy:", acc)
print("EM score:", em_score)
print("F1 score:", f1_score)

print("\n\nT/P\tanswer_start\tanswer_end\n")
for i in range(len(start_true)):
  pred_span = tokenizer.decode(input_ids[i][start_pred[i]:end_pred[i]+1], skip_special_tokens=True)
  true_span = tokenizer.decode(input_ids[i][start_true[i]:end_true[i]+1], skip_special_tokens=True)
  print(f"true\t{start_true[i]}\t{end_true[i]}\t{true_span}\n"
        f"pred\t{start_pred[i]}\t{end_pred[i]}\t{pred_span}\n")


100%|██████████| 133/133 [01:06<00:00,  2.01it/s]



Accuracy: 0.699953007518797
EM score: 0.5723746452223274
F1 score: 0.7658647335965703


T/P	answer_start	answer_end

true	42	45	february 1, 2016
pred	42	45	february 1, 2016

true	144	144	1992
pred	144	144	1992

